<a href="https://colab.research.google.com/github/marlo37/Pre_trained_model_train/blob/main/batch_size_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvcc -V
!pip3 install torch torchvision

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using 

Source: https://github.com/dandiws/CNN-MNIST-pytorch/blob/master/cnn_mnist.py

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np

from PIL import Image
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
def main(batch_size):
    torch.manual_seed(2)
    torch.cuda.manual_seed_all(2)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    def im_convert(tensor):
        image = tensor.to("cpu").clone().detach()
        image = image.numpy().squeeze()
        return image

    class Net(nn.Module):
        def __init__(self):
            super(Net,self).__init__()
            self.conv1 = nn.Conv2d(1,10,kernel_size=5,stride=1)
            self.conv2 = nn.Conv2d(10,10,kernel_size=5,stride=1)
            self.pool = nn.MaxPool2d(kernel_size=2,stride=2) #2x2 maxpool
            self.fc1 = nn.Linear(4*4*10,100)
            self.fc2 = nn.Linear(100,10)

        def forward(self,x):
            x = F.relu(self.conv1(x)) #24x24x10
            x = self.pool(x) #12x12x10
            x = F.relu(self.conv2(x)) #8x8x10
            x = self.pool(x) #4x4x10
            x = x.view(-1, 4*4*10) #flattening
            x = F.relu(self.fc1(x))
            x = self.fc2(x)
            return x

    train_ds = datasets.MNIST('../data',train=True,download=True, transform=transforms.Compose([transforms.ToTensor()]))

    validation_split = .1
    shuffle_dataset = True
    random_seed= 2

    # Creating data indices for training and validation splits:
    dataset_size = len(train_ds)
    indices = list(range(dataset_size))
    split = int(np.floor(validation_split * dataset_size))
    if shuffle_dataset :
        np.random.seed(random_seed)
        np.random.shuffle(indices)
    train_indices, val_indices = indices[split:], indices[:split]
    # Creating PT data samplers and loaders:
    train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
    valid_sampler = torch.utils.data.SubsetRandomSampler(val_indices)


    train_loader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size,
                                            sampler=train_sampler)
    validation_loader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size,
                                                    sampler=valid_sampler)

    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data',train=False,download=True,
        transform=transforms.Compose([transforms.ToTensor()])),batch_size=batch_size,shuffle=True)

    # fig, ax = plt.subplots(nrows=2,ncols=3)

    # i=0
    # for row in ax:
    #   for col in row:
    #     col.imshow(im_convert(train_loader.dataset[i][0]))
    #     col.set_title("digit "+str(train_loader.dataset[i][1]))
    #     col.axis("off")
    #     i+=1

    model = Net().cuda()
    optimizer = optim.SGD(model.parameters(),lr=0.01)
    criterion = nn.CrossEntropyLoss()

    train_errors = []
    train_acc = []
    val_errors = []
    val_acc = []
    n_train = len(train_loader)*batch_size
    n_val = len(validation_loader)*batch_size
    import time
    start_time = time.time()
    for i in range(10):
        total_loss = 0
        total_acc = 0
        c = 0
        for images,labels in train_loader:
            images = images.cuda()
            labels = labels.cuda()

            optimizer.zero_grad()
            output = model(images)
            loss = criterion(output,labels)
            loss.backward()
            optimizer.step()

            total_loss+=loss.item()
            total_acc+=torch.sum(torch.max(output,dim=1)[1]==labels).item()*1.0
            c+=1


        #validation

        total_loss_val = 0
        total_acc_val = 0
        c = 0
        for images,labels in validation_loader:
            images = images.cuda()
            labels = labels.cuda()
            output = model(images)
            loss = criterion(output,labels)

            total_loss_val +=loss.item()
            total_acc_val +=torch.sum(torch.max(output,dim=1)[1]==labels).item()*1.0
            c+=1

        train_errors.append(total_loss/n_train)
        train_acc.append(total_acc/n_train)
        val_errors.append(total_loss_val/n_val)
        val_acc.append(total_acc_val/n_val)
    end_time = time.time()-start_time

    """
    print("Trainig complete")


    fig, ax = plt.subplots(nrows=1, ncols=2)
    ax[0].plot(train_errors, 'r',label="Train")
    ax[0].plot(val_errors, 'g', label="Validation")
    ax[0].set_title("Grafik error")
    ax[0].set_ylabel("Error (cross-entropy)")
    ax[0].set_xlabel("Epoch")
    ax[0].legend()
    ax[1].plot(train_acc, 'r',label="Train")
    ax[1].plot(val_acc, 'g', label="Validation")
    ax[1].set_title("Grafik akurasi")
    ax[1].set_ylabel("Accuracy")
    ax[1].set_xlabel("Epoch")
    ax[1].legend()
    plt.show()
    """

    total_acc = 0
    for images,labels in test_loader:
        images = images.cuda()
        labels = labels.cuda()
        output = model(images)
        total_acc+=torch.sum(torch.max(output,dim=1)[1]==labels).item()*1.0

    print(f"Batch size {batch_size} Time required {end_time}s :",total_acc/len(test_loader.dataset))

In [3]:
main(256)
main(128)
main(64)
main(32)
main(16)
main(8)
main(4)
main(2)
main(1)



100%|██████████| 9912422/9912422 [00:00<00:00, 277964533.76it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 39342544.28it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 234604002.73it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw




100%|██████████| 4542/4542 [00:00<00:00, 6326977.34it/s]


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw

Batch size 256 Time required 66.33616018295288s : 0.9414
Batch size 128 Time required 70.93315362930298s : 0.9669
Batch size 64 Time required 79.03622579574585s : 0.978
Batch size 32 Time required 95.71692824363708s : 0.9848
Batch size 16 Time required 144.3849356174469s : 0.9858
Batch size 8 Time required 199.46242308616638s : 0.9845
Batch size 4 Time required 329.23670983314514s : 0.9846
Batch size 2 Time required 590.1485893726349s : 0.9878
Batch size 1 Time required 1100.3400127887726s : 0.9717
